In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the pixel values between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, 100)
y_test = keras.utils.to_categorical(y_test, 100)


In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'D:\Underwater_Image\Sea Animals\archive'

# Initialize empty lists for images and labels
X_train = []
Y_train = []

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Loop through subdirectories (classes)
for class_folder in os.listdir(directory_path):
    class_path = os.path.join(directory_path, class_folder)
    class_label = class_folder
    
    # Encode class labels
    encoded_label = label_encoder.fit_transform([class_label])[0]

    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpg'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(encoded_label)

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


X_train shape: (13666, 32, 32, 3)
Y_train shape: (13666,)


In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
import glob

target_size = (32, 32)  # Change the values as per your requirement
# Load the pre-trained ResNet50 model with modified input shape
model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(target_size[0], target_size[1], 3))

In [3]:
import pandas as pd
# Define the file path
file_path = r"D:\21k_avg_feature.csv"
# Read the tab-separated CSV file into a DataFrame
df = pd.read_csv(file_path, delimiter='\t')
columns_to_drop = [df.columns[0], df.columns[-1]]
data = df.drop(columns_to_drop, axis=1)
# Display the head of the DataFrame
data.head(1)

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,...,Feature 2038,Feature 2039,Feature 2040,Feature 2041,Feature 2042,Feature 2043,Feature 2044,Feature 2045,Feature 2046,Feature 2047
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.037739,0.0,0.0,0.0,0.0,3.016108,0.0,0.0


In [7]:
np.shape(X_train)

(9590, 32, 32, 3)

In [4]:
from PIL import Image
import numpy as np
ft = model.predict(np.array(x_train).astype("float32"))

1563/1563 [==============================] - 27s 13ms/step


In [5]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 400
batch_size = 100
max_iter = 100

kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, max_iter=max_iter)
kmeans.fit(ft)
# Retrieve the cluster centers
ct = kmeans.cluster_centers_.tolist()

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\sklearn\cluster\_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [6]:
from tqdm import tqdm

combo_list = []  # Initialize combo_list
tot_dist = []
# Iterate over ft
for i in tqdm(range(len(ct))):
    distances = []

    # Calculate distances for each row in data
    for index, row in data.iterrows():
        row_array = row.to_numpy()  # Convert row to numpy array
        distance = np.linalg.norm(ct[i] - row_array)  # Calculate Euclidean distance
        distances.append(distance)

    tot_dist.append(distances)
print(np.shape(tot_dist))
#new_shape = (1000, 30)
#re_dist = np.transpose(tot_dist, (1, 0))
re_dist = tot_dist
print(np.shape(re_dist))

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [16:14<00:00,  2.44s/it]


(400, 10450)
(400, 10450)


In [ ]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\dist_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(re_dist, file)

print("Combo list saved successfully.")

In [2]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\index_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    index_list = pickle.load(file)

print("Combo list loaded successfully.")
print(index_list)


Combo list loaded successfully.
[90, 103, 547, 1040, 1053, 1497, 2040, 2497, 3053, 3497, 4053, 4497, 5497, 6092, 6099, 6106, 6107, 6108, 6126, 6132, 6447, 6450, 6464, 6492, 6997, 7592, 7599, 7606, 7607, 7608, 7626, 7632, 7947, 7950, 7964, 7984, 7992, 8497, 9092, 9099, 9106, 9107, 9108, 9126, 9132, 9447, 9450, 9464, 9492, 9997]


In [7]:
df.columns

Index(['Directory', 'Feature 0', 'Feature 1', 'Feature 2', 'Feature 3',
       'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7', 'Feature 8',
       ...
       'Feature 2039', 'Feature 2040', 'Feature 2041', 'Feature 2042',
       'Feature 2043', 'Feature 2044', 'Feature 2045', 'Feature 2046',
       'Feature 2047', 'Unnamed: 2049'],
      dtype='object', length=2050)

In [3]:
selected_data = df.loc[index_list, ['Directory']].to_numpy().flatten()

In [4]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\combo_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(selected_data, file)

print("Combo list saved successfully.")


Combo list saved successfully.


In [5]:
import pickle

# Loading the list from the pickle file
file_path = r"D:\combo_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    loaded_combo_list = pickle.load(file)

print("Combo list loaded successfully.")
print(loaded_combo_list)


Combo list loaded successfully.
['F:/imagenet21k_resized/imagenet21k_train\\n00446311\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n00448748\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01596608\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n00446311\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n00448748\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01596608\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n00446311\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01596608\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n00448748\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01596608\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n00448748\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01596608\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01596608\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01943899\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01946630\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01953361\\'
 'F:/imagenet21k_resized/imagenet21k_train\\n01953594\\'